<a href="https://colab.research.google.com/github/aashitadutta/FinancialNarrativeSummarization/blob/main/Summarization_using_BERT_and_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pip install transformers==2.6.0

import transformers
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel, DistilBertModel, DistilBertTokenizer, BertTokenizer, BertForTokenClassification
import numpy as np

import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
%tensorflow_version 1.x
from keras.preprocessing.sequence import pad_sequences

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import json
import matplotlib.pyplot as plt
import timeit
import torch
import textwrap
wrapper = textwrap.TextWrapper(width=70)
SEED = 1234
torch.manual_seed(SEED)

     |████████████████████████████████| 542kB 6.7MB/s 
     |████████████████████████████████| 901kB 13.6MB/s 
     |████████████████████████████████| 133kB 33.7MB/s 
     |████████████████████████████████| 5.6MB 11.4MB/s 
     |████████████████████████████████| 1.2MB 45.4MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 7.7MB 40.4MB/s 
ERROR: botocore 1.20.101 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
!git lfs install
!git clone 'https://huggingface.co/distilbert-base-uncased'
!git clone 'https://huggingface.co/bert-base-cased'
!git clone 'https://huggingface.co/bert-base-uncased'
!git clone 'https://huggingface.co/gpt2'

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'distilbert-base-uncased'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 44 (delta 16), reused 0 (delta 0)
Unpacking objects: 100% (44/44), done.
Cloning into 'bert-base-cased'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 61 (delta 23), reused 0 (delta 0)
Unpacking objects: 100% (61/61), done.
Cloning into 'bert-base-uncased'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 52 (delta 19), reused 0 (delta 0)
Unpacking objects: 100% (52/52), done.
Cloning into 'gpt2'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100

In [ ]:

BERT_pretrained_weights = 'distilbert-base-uncased' 


BERTforTokenClassification_config_directory = 'bert-base-cased' 
token_label_files = 'bert-base-cased/tokenizer.json' 

GPT2_config_directory = 'gpt2'

print('which BERT pre-trained ? ',BERT_pretrained_weights)
print('where is BERT token classifier dir ? ',BERTforTokenClassification_config_directory)
print('where is GPT2 dir ? ',GPT2_config_directory)

which BERT pre-trained ?  distilbert-base-uncased
where is BERT token classifier dir ?  bert-base-cased
where is GPT2 dir ?  gpt2


In [ ]:
print('----loading pre-trained BERT----')
BERT_pretrained = DistilBertModel.from_pretrained(BERT_pretrained_weights)
tokenizer_pretrained = DistilBertTokenizer.from_pretrained(BERT_pretrained_weights)
print('----loading token labels----')
with open(token_label_files, 'r') as fp:
    POS2idx = json.load(fp)

POS_values = list(POS2idx.keys())
print('----loading BERT token classifier----')
BERT_token_classifier = BertForTokenClassification.from_pretrained(
                      BERTforTokenClassification_config_directory)

tokenizer_token_classifier = BertTokenizer.from_pretrained(
                      BERTforTokenClassification_config_directory)

print('----loading GPT2 summary generator----')
tokenizer_GPT2 = GPT2Tokenizer.from_pretrained(
                  GPT2_config_directory)
special_tokens = {'bos_token':'<|startoftext|>','eos_token':'<|endoftext|>','pad_token':'<pad>','additional_special_tokens':['<|keyword|>','<|summarize|>']}
tokenizer_GPT2.add_special_tokens(special_tokens)
GPT2_generator = GPT2DoubleHeadsModel.from_pretrained(
                  GPT2_config_directory)

----loading pre-trained BERT----



----loading token labels----
----loading BERT token classifier----



----loading GPT2 summary generator----


In [ ]:
import os

In [ ]:
max_len =  512
i=0
path = "/content/drive/MyDrive/lstm-input"
path1="/content/drive/MyDrive/lstm-summ"
files = os.listdir(path)
files=sorted(files)
files2=os.listdir(path1)
files2=sorted(files2)
print(files)
print(files2)
d=0;
for (f1,f2) in zip(files,files2):
  if d==10:
    break
  d=d+1;
  f1_score1=[]
  precision1=[]
  recall1=[]
  f1_score2=[]
  precision2=[]
  recall2=[]
  f1_scoreL=[]
  precisionL=[]
  recallL=[]
  if f1.endswith(".txt") and f2.endswith(".txt"):
    file_path = f"{path}/{f1}"
    name1=str(f1)
    #with open(file_path,'r+') as file:
    #  file1 = file.read().replace('\n', '')
    #file1=open(file_path,'r+').replace('\n',' ')
    file1 = open(file_path, "r+")
    text=file1.read()
    file1.close()
    file1=open(file_path, "w")
    text2=''
    if '\n' in text:
      text2 = text.replace('\n' , ' ')
      #print(text2)
      file1.write(text2)
    file1.truncate(160000)
    file1.seek(0, os.SEEK_END)
    print("Size of file1 is :", file1.tell(), "bytes")
    print(file1)
    file1.close()


    ##file2##
    file_path = f"{path1}/{f2}"
    name1=str(f2)
    #file2=open(file_path,'r+').replace('\n',' ')
    file2 = open(file_path, "r+")
    text=file2.read()
    file2.close()
    file2=open(file_path, "w")
    text2=''
    if '\n' in text:
      text2 = text.replace('\n' , ' ')
      #print(text2)
      file2.write(text2)
    file2.truncate(160000)
    file2.seek(0, os.SEEK_END)
    #print("Size of file2 is :", file2.tell(), "bytes")
    #print(file2)
    file2.close()
    #for line in file2:
    #  print(line)
    #input_text = file1
    #print(len(input_text))
    #reference_text=file2.read().replace('\n',' ')
    file1 = open(file_path, "r+")
    text=file1.read()
    file2 = open(file_path, "r+")
    reference_text=file2.read()
    #print(text)
    paragraph_split =sent_tokenize(input_text)

    print('input text has',len(paragraph_split) ,'sentences.')
    file1.close()
    print('tokenizing sentences')
   
    path = "/content/drive/MyDrive"
    os.chdir(path)
    input_tokens = []
    for i in paragraph_split:
      input_tokens.append(tokenizer_pretrained.encode(i, 
                              add_special_tokens=True,truncation=True))
    temp = []
    for i in input_tokens:
     temp.append(len(i))
    input_ids = pad_sequences(input_tokens, 
                          maxlen=max_len, dtype="long", 
                          value=0, 
                          truncating="post", 
                          padding="post")

    print('creating attention masks')

    attention_masks = []
    for sent in input_ids:
      att_mask = [int(token_id > 0) for token_id in sent]  
      attention_masks.append(att_mask)  

    input_ids = torch.tensor(input_ids)  
    attention_mask = torch.tensor(attention_masks)
    number_extract = 2 
    
    input_ids = input_ids.to('cpu')
    BERT_pretrained = BERT_pretrained.to('cpu')
    lattention_mask = attention_mask.to('cpu')

    with torch.no_grad():
     last_hidden_states = BERT_pretrained(input_ids, 
                             attention_mask=attention_mask)

    sentence_features = last_hidden_states[0][:,0,:].detach().cpu().numpy()

    print('performing k-medoid clustering with '
        ,number_extract,' clusters')

    kmeans = KMeans(n_clusters=number_extract,random_state=0).fit(sentence_features)
    cluster_center = kmeans.cluster_centers_
    nbrs = NearestNeighbors(n_neighbors= 1,algorithm='brute').fit(sentence_features)
    distances, indices = nbrs.kneighbors(cluster_center.reshape(number_extract,-1))

    rows, cols = (number_extract, len(indices[0]))
    ind =np.array([[0]*cols]*rows)
    dis=-1*distances
    xord = dis.argsort()
    j=0
    i=0  
    for i in range(rows):
     for j in range(cols):
      ind[i][j]=indices[i][xord[i][j]]
    topic_answer = []
    j=0
    ind=ind[ind[:, 0].argsort()]
    print(ind)
    while j!=number_extract:
     i=ind[j][0]
     topic_answer.append(paragraph_split[i])
     j=j+1
    if j==number_extract:
      break


    print('result:')

    print('the ',number_extract,' extracted sentences are')
    for i in topic_answer:
     print(i)

    topic_answer_string = ''
    for topic in topic_answer:
      topic_answer_string = topic_answer_string + ' '+ topic
    scores = rouge.get_scores(topic_answer_string, reference_text)
    scores_rouge1 = [score['rouge-1']['r'] for score in scores]
    scores_rouge2 = [score['rouge-2']['r'] for score in scores]
    scores_rougeL = [score['rouge-l']['r'] for score in scores]                 
    recall1.append(scores_rouge1[0])
    recall2.append(scores_rouge2[0])
    recallL.append(scores_rougeL[0])
    scores_rouge1 = [score['rouge-1']['p'] for score in scores]
    scores_rouge2 = [score['rouge-2']['p'] for score in scores]
    scores_rougeL = [score['rouge-l']['p'] for score in scores]                 
    precision1.append(scores_rouge1[0])
    precision2.append(scores_rouge2[0])
    precisionL.append(scores_rougeL[0])
    scores_rouge1 = [score['rouge-1']['f'] for score in scores]
    scores_rouge2 = [score['rouge-2']['f'] for score in scores]
    scores_rougeL = [score['rouge-l']['f'] for score in scores]                 
    f1_score1.append(scores_rouge1[0])
    f1_score2.append(scores_rouge2[0])
    f1_scoreL.append(scores_rougeL[0])
    completeName=os.path.join("/content/drive/MyDrive/output1", str(name1))
    file1 = open(completeName, "w")
    file1.write(topic_answer_string)
    file1.close()
   










 

['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt', '64.txt', '65.txt', '66.txt', '67.txt', '72.txt', '73.txt', '92.txt', '93.txt']
['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt'

ValueError: ignored

In [ ]:
 """distances, indices = nbrs.kneighbors(cluster_center.reshape(number_extract,-1))
    indices
    topic_answer_string
    print(os.path)"""